In [57]:
import pandas as pd
df = pd.read_csv("Names.csv")

In [58]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

import os
from secretkey import api_key

os.environ["OPENAI_API_KEY"]= api_key

chat =  ChatOpenAI(temperature=0.7,model="gpt-4-0125-preview")

In [59]:
for index, row in df.iterrows():
    answer = chat([HumanMessage(content=f"""You will help me create training dataset for generating text to text prompt:

        User_prompt:
        Generate Indian kids name based on zodiac sign Aries and it should be made from two words Jay and Shreya

        Answer:
        Boys:
        Jayansh: Combining "Jay" for victory and "Ansh" which means part, suggesting a victorious part of life.
        Jayveer: Merging "Jay" with "Veer" (brave), indicating a victorious and brave individual.
        Shreyansh: Integrating "Shreya" for luck with "Ansh", portraying a lucky and significant part of life.
        Jayakar: Combining "Jay" with "Akar", which means shape or form, symbolizing a victorious form.
        Shreyaratna: Merging "Shreya" with "Ratna" (jewel), suggesting a precious and lucky jewel.
        Girls:
        Jayshree: A blend of "Jay" and "Shree" (an alternative spelling of Shreya), symbolizing victory and auspiciousness.
        Shreyajay: This combines "Shreya" and "Jay", reversing the order to focus on auspicious victory.
        Jayshriya: Integrating "Jay" with a twist on "Shreya", indicating a victorious and auspicious individual.
        Shreyasree: Merging "Shreya" with "Sree" (a variant of Shreya), enhancing the auspicious and lucky connotations.
        Jayashree: Combining "Jaya", a variant of "Jay", with "Shree", denoting a victorious and auspicious person.

        User_prompt:
        Generate Indian kids name based on zodiac sign Aries and it should be made from two words Viraj and Neha

        Answer:
        Boys:
        Virajit: Combining "Viraj" with the suffix "it," which can mean one who possesses, suggesting one who possesses brilliance or sovereignty.
        Nehvir: A fusion of "Neha" and a part of "Viraj," implying love and brilliance together.
        Virnehal: Integrating "Vir" from "Viraj" and "Nehal" from "Neha," symbolizing a sovereign filled with love.
        Rajneha: Merging "Raj" from "Viraj" with "Neha," indicating a royal or sovereign love.
        Vineha: A blend of "Vi" from "Viraj" and "Neha," creating a unique name that hints at both brilliance and affection.
        Girls:
        Neharika: Inspired by "Neha" and incorporating "Rika" from "Viraj" backwards, symbolizing a dewdrop, which combines the ideas of love with a celestial sparkle.
        Vinehaa: Similar to the boys' version but with an extra "a" for a softer sound, blending "Viraj" and "Neha" to imply brilliance and love.
        Rajnehaa: A variation of "Rajneha" for girls, with an extra "a" to feminize the name, denoting royal love.
        Nehviraj: Flipping "Nehvir" for a girl, emphasizing the sovereignty and brilliance associated with love.
        Virajneha: Keeping both names intact but combining them, to suggest the sovereign brilliance of love.

        User_prompt:
        Generate Indian kids name based on zodiac sign Aries and it should be made from two words {row.iloc[0]} and {row.iloc[1]}""")])

    df.at[index, "Answer"]= answer.content

Preparing training data

Option 1:

In [70]:
import json
json_objects = []

for index, row in df.iterrows():
    # Construct the JSON object for each row
    json_obj = {
        "messages":[
            {
                "role": "system",
                "content": "You are an astrology guru who have mastered in suggesting names for the new born children. You can combine two names and generate two meaningful names for a boy and a girl."
            },
            {
                "role": "user",
                "content": f"name1={row['Name1']},name2={row['Name2']}"
            },
            {
                "role": "assistant",
                "content": row['Answer']
            }
        ]
    }
    
    # Add the JSON object to our list
    json_objects.append(json_obj)

# Now, write the JSON objects to a file, one per line
with open("training_data.jsonl", "w", encoding='utf-8') as file:
    for obj in json_objects:
        file.write(json.dumps(obj, separators=(',', ':')) + '\n')

Option 2:

In [52]:
training_data = pd.DataFrame()
for index, row in df.iterrows():
    instance = f"""{{"messages": [{{"role": "system", "content": "You are an astrology guru who have mastered in suggesting names for the new born children. You can combine two names and generate two meaningful names for a boy and a girl."}},{{"role": "user", "content": "name1={row.iloc[0]},name2={row.iloc[1]}"}},{{"role": "assistant", "content": "{row.iloc[2]}"}}]}}"""
    training_data.at[index, "Answer"]=instance

In [54]:
training_data["Answer"].to_csv("training_data.csv",index=False)
data = pd.read_csv("training_data.csv")
data.to_json("training_data.jsonl")

In [55]:
import csv
import json

# Open the CSV file for reading
with open('training_data.csv', mode='r', encoding='utf-8') as csv_file:
    # Create a CSV reader object
    csv_reader = csv.DictReader(csv_file)
    
    # Open the JSONL file for writing
    with open('training_data.jsonl', mode='w', encoding='utf-8') as jsonl_file:
        # Iterate over each row in the CSV file
        for row in csv_reader:
            # Convert the row (which is already a dictionary) to a JSON string
            json_str = json.dumps(row)
            # Write the JSON string to the JSONL file, followed by a newline character
            jsonl_file.write(json_str + '\n')